import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

import and modify dataset

In [ ]:
import pandas as pd

# Load the CSV file
input_csv_file = '/content/Restaurant_reviews.csv'  # Update with your CSV file path
df = pd.read_csv(input_csv_file)
df = df.drop(index=7601)
df['Review'] = df['Review'].astype(str).agg(''.join)

# Create the new DataFrame with 'review' and 'liked' columns
dataset = df[['Review']].copy()
df['Rating'] = pd.to_numeric(df['Rating'])
dataset['Liked'] = df['Rating'].apply(lambda x: 1 if x > 2 else 0)
print(dataset['Liked'].value_counts())
dataset.head()




Liked
1    7533
0    2466
Name: count, dtype: int64


<ipython-input-34-731690c88042>:7: FutureWarning: using <built-in method join of str object at 0x7de919ea8030> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df['Review'] = df['Review'].astype(str).agg(''.join)


,Review,Liked
0,"The ambience was good, food was quite good . h...",1
1,Ambience is too good for a pleasant evening. S...,1
2,A must try.. great food great ambience. Thnx f...,1
3,Soumen das and Arun was a great guy. Only beca...,1
4,Food is good.we ordered Kodi drumsticks and ba...,1


cleaning the text

In [ ]:
nlp = spacy.load("en_core_web_sm")



negation_words = {
    'not', 'never', 'no', 'nothing', 'nowhere', 'none', 'nobody',
    'neither', 'nor', 'cannot', 'isn\'t', 'aren\'t', 'wasn\'t', 'weren\'t',
    'won\'t', 'wouldn\'t', 'shouldn\'t', 'can\'t', 'don\'t', 'doesn\'t',
    'didn\'t', 'hasn\'t', 'haven\'t', 'hadn\'t', 'couldn\'t', 'mightn\'t', 'mustn\'t'
}

def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    doc = nlp(text)

    lemmatized_tokens = [
        token.lemma_ for token in doc
        if not token.is_stop or not token.is_punct or token.text in negation_words
    ]
    result = ' '.join(lemmatized_tokens)
    result = re.sub('\s+', ' ', result).strip()
    return result



corpus = [preprocess_text(review) for review in dataset['Review']]
print(corpus)

['the ambience be good food be quite good have saturday lunch which be cost effective good place for a sate brunch one can also chill with friend and or parent waiter soumen das be really courteous and helpful', 'ambience be too good for a pleasant evening service be very prompt food be good over all a good experience souman das kudo to the service', 'a must try great food great ambience thnx for the service by pradeep and subroto my personal recommendation be penne alfredo pasta also the music in the background be amazing', 'soumen das and arun be a great guy only because of their behavior and sincerety and good food off course I would like to visit this place again', 'food be good we order kodi drumstick and basket mutton biryani all be good thank to pradeep he serve well we enjoy here ambience be also very good', 'ambiance be good service be good food be apradeecp and subro good service food be good papiya good hostess and ur caption very good this be star restaurant', 'its a very n

# neural networks architecures for word vectors, bag of words.

create neural network architecture to be used later to train on the word vectors




In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2


model = Sequential()


model.add(Dense(128, input_shape=(300,), activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.6))

model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.6))

model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.6))


model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



create neural network architecture to be used later to train on the bag of words vectors.

In [ ]:

model_bag = Sequential()


model_bag.add(Dense(128, input_shape=(1400,), activation='relu', kernel_regularizer=l2(0.01)))
model_bag.add(Dropout(0.6))


model_bag.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model_bag.add(Dropout(0.6))


model_bag.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model_bag.add(Dropout(0.6))


model_bag.add(Dense(1, activation='sigmoid'))


model_bag.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Text classification using Bag of Words for vectorization, implemented with Naive Bayes and neural network models.

using bag of words model to vectorise sentences.




In [ ]:
cv = CountVectorizer(max_features=1400)



X_bag = cv.fit_transform(corpus).toarray()
y_bag = dataset['Liked']

print(len(cv.get_feature_names_out()))

print(y_bag.shape)
X_bag.shape


1400
(9999,)


(9999, 1400)

split the dataset.

In [ ]:
X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(X_bag, y_bag, test_size = 0.20, random_state = 0)


train naive bayes.

In [ ]:
gaussian_classifier = GaussianNB()
gaussian_classifier.fit(X_train_bag, y_train_bag)

GaussianNB()

evaluate gaussian classifier.

In [ ]:
y_pred_bag = gaussian_classifier.predict(X_test_bag)
cm = confusion_matrix(y_test_bag, y_pred_bag)
print(cm)
accuracy_score(y_test_bag, y_pred_bag)

[[418  53]
 [730 799]]


0.6085

train the neural network.


In [ ]:
history_bag = model_bag.fit(X_bag, y_bag, epochs=25, batch_size=32, validation_split=0.2)


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7385 - loss: 2.2305 - val_accuracy: 0.8100 - val_loss: 0.7403
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8694 - loss: 0.6176 - val_accuracy: 0.8665 - val_loss: 0.5052
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8991 - loss: 0.4545 - val_accuracy: 0.8610 - val_loss: 0.4604
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9030 - loss: 0.4114 - val_accuracy: 0.8775 - val_loss: 0.4398
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9050 - loss: 0.4016 - val_accuracy: 0.8705 - val_loss: 0.4372
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9000 - loss: 0.3991 - val_accuracy: 0.8665 - val_loss: 0.4339
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9094 - loss: 0.3887 - val_accuracy: 0.8625 - val_loss: 0.4274
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9102 - loss: 0.3854 - val_accuracy: 0.

# Text classification using pretrained glove for vectorization, implemented with Naive Bayes and neural network models.

create glove

In [ ]:
import gensim.downloader as api

wv_glove = api.load('glove-wiki-gigaword-300')




[==================================================] 100.0% 376.1/376.1MB downloaded


using glove model to vectorise sentences.

In [ ]:
num_of_words_oov = 0

def get_review_vector(review, embedding_model):
    global num_of_words_oov
    words = review.lower().split()
    word_vectors = []

    for word in words:
        try:
            vector = embedding_model[word]
            word_vectors.append(vector)
        except KeyError:
            num_of_words_oov += 1

    if len(word_vectors) == 0:
        return np.zeros(embedding_model.vector_size)

    return np.mean(word_vectors, axis=0)

X_glove = np.array([get_review_vector(review, wv_glove) for review in corpus])
y_glove = dataset['Liked']
print(num_of_words_oov)
X_glove.shape

8364


(9999, 300)

split the dataset.

In [ ]:
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(X_glove, y_glove, test_size = 0.20, random_state = 0)


train navive bayes.

In [ ]:
gaussian_classifier = GaussianNB()
gaussian_classifier.fit(X_train_glove, y_train_glove)

GaussianNB()

evaluate gaussian classifier.

In [ ]:
y_pred_glove = gaussian_classifier.predict(X_test_glove)
cm = confusion_matrix(y_test_glove, y_pred_glove)
print(cm)
accuracy_score(y_test_glove, y_pred_glove)

[[ 422   49]
 [ 388 1141]]


0.7815

train the neural network.

In [ ]:
history_glove = model.fit(X_glove, y_glove, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7032 - loss: 2.2339 - val_accuracy: 0.7135 - val_loss: 0.7103
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7637 - loss: 0.6019 - val_accuracy: 0.7135 - val_loss: 0.5555
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7742 - loss: 0.5001 - val_accuracy: 0.8345 - val_loss: 0.5190
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8381 - loss: 0.4604 - val_accuracy: 0.8460 - val_loss: 0.5072
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8473 - loss: 0.4482 - val_accuracy: 0.8440 - val_loss: 0.4892
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8600 - loss: 0.4431 - val_accuracy: 0.8365 - val_loss: 0.4981
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8634 - loss: 0.4329 - val_accuracy: 0.8515 - val_loss: 0.5027
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8605 - loss: 0.4277 - val_accuracy: 0.

# Text classification using non pre-trained fasttext vectorization, implemented with Naive Bayes and neural network models.

install gensim

In [ ]:
!pip install gensim

create fasttext model and train it on the data

In [ ]:
from gensim.models import FastText
from nltk.tokenize import word_tokenize

fasttext_model = FastText(
    sentences=corpus,
    vector_size=300,
    window=5,
    min_count=1,
    epochs=10
)


tokenized_corpus = [word_tokenize(review) for review in corpus]

X_fasttext = np.array([
    np.mean([fasttext_model.wv[word] for word in review if word in fasttext_model.wv], axis=0)
    if [fasttext_model.wv[word] for word in review if word in fasttext_model.wv]
    else np.zeros(fasttext_model.vector_size)
    for review in tokenized_corpus
])


y_fasttext = dataset['Liked']

X_fasttext.shape



(9999, 300)

split the dataset created.

In [ ]:
X_train_fasttext, X_test_fasttext, y_train_fasttext, y_test_fasttext = train_test_split(X_fasttext, y_fasttext, test_size = 0.20, random_state = 0)


train gaussian.

In [ ]:
gaussian_classifier = GaussianNB()
gaussian_classifier.fit(X_train_fasttext, y_train_fasttext)

GaussianNB()

evaluate gaussian classifier.

In [ ]:
y_pred_fasttext = gaussian_classifier.predict(X_test_fasttext)
cm = confusion_matrix(y_test_fasttext, y_pred_fasttext)
print(cm)
accuracy_score(y_test_fasttext, y_pred_fasttext)

[[ 416   55]
 [1177  352]]


0.384

train the neural network




In [ ]:
history_fasttext = model.fit(X_fasttext, y_fasttext, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7470 - loss: 0.6635 - val_accuracy: 0.7135 - val_loss: 0.6287
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7559 - loss: 0.5966 - val_accuracy: 0.7135 - val_loss: 0.6146
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7634 - loss: 0.5780 - val_accuracy: 0.7135 - val_loss: 0.6115
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7567 - loss: 0.5756 - val_accuracy: 0.7135 - val_loss: 0.6071
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7524 - loss: 0.5737 - val_accuracy: 0.7135 - val_loss: 0.6072
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7588 - loss: 0.5649 - val_accuracy: 0.7135 - val_loss: 0.6062
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7659 - loss: 0.5539 - val_accuracy: 0.7135 - val_loss: 0.6056
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7650 - loss: 0.5552 - val_accuracy: 0.

# Text classification using bidirectional lstm for vectorization, and prediction.

tokenize the text, add padding.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(corpus)


X_lstm = tokenizer.texts_to_sequences(corpus)

X_lstm = pad_sequences(X_lstm, maxlen=100)

y_lstm = dataset['Liked']


create lstm architecutre

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


model_lstm = Sequential()


model_lstm.add(Embedding(input_dim=2000, output_dim=128, input_length=50))

model_lstm.add(Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.05))))


model_lstm.add(LSTM(32, return_sequences=False, kernel_regularizer=l2(0.05)))


model_lstm.add(Dropout(0.6))


model_lstm.add(Dense(32, activation='relu'))


model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


train the model

In [ ]:

history_lstm = model_lstm.fit(X_lstm, y_lstm, epochs=15, batch_size=32, validation_split=0.2)


Epoch 1/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 50s 177ms/step - accuracy: 0.7727 - loss: 9.5011 - val_accuracy: 0.7175 - val_loss: 0.4807
Epoch 2/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 43s 173ms/step - accuracy: 0.8549 - loss: 0.3599 - val_accuracy: 0.8720 - val_loss: 0.3360
Epoch 3/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 45s 181ms/step - accuracy: 0.9131 - loss: 0.2572 - val_accuracy: 0.8700 - val_loss: 0.3118
Epoch 4/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 80s 174ms/step - accuracy: 0.9342 - loss: 0.2060 - val_accuracy: 0.8695 - val_loss: 0.3181
Epoch 5/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 174ms/step - accuracy: 0.9337 - loss: 0.2034 - val_accuracy: 0.8780 - val_loss: 0.3145
Epoch 6/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 173ms/step - accuracy: 0.9514 - loss: 0.1643 - val_accuracy: 0.8635 - val_loss: 0.3736
Epoch 7/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 90s 207ms/step - accuracy: 0.9435 - loss: 0.1732 - val_accuracy: 0.8720 - val_loss: 0.3316
Epoch 8/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 64s 253ms/step - accuracy: 0.9524 - loss: 0